# Word Embeddings in SQLite3

This example uses the SQLite3 Connector within Python3 to store and retrieve various amounts of Word Embeddings.

In [17]:
import sqlite3
import io
import time
import numpy
import plotly

# Dummy Embeddings

For testing purposes we will use randomly generated numpy arrays as dummy embbeddings. Since loading gigabytes of gensim KeyedVectors is slow and annoying.

In [18]:
def embeddings(n=1000, dim=300):
    """
    Yield n tuples of random numpy arrays of *dim* length indexed by *n*
    """
    idx = 0
    while idx < n:
        yield (str(idx), numpy.random.rand(dim))
        idx += 1

# SQLite Adapders

In [19]:
def adapt_array(array):
    """
    Using the numpy.save function to save a binary version of the array,
    and BytesIO to catch the stream of data and convert it into a sqlite3.Binary.
    """
    out = io.BytesIO()
    numpy.save(out, array)
    out.seek(0)

    return sqlite3.Binary(out.read())

def convert_array(text):
    """
    Using BytesIO to convert the binary version of the array back into a numpy array.
    """
    out = io.BytesIO(text)
    out.seek(0)

    return numpy.load(out)

# Register the new adapters
sqlite3.register_adapter(numpy.ndarray, adapt_array)
sqlite3.register_converter('array', convert_array)

In [20]:
connection = sqlite3.connect(':memory:', detect_types=sqlite3.PARSE_DECLTYPES)
cursor = connection.cursor()
cursor.execute('CREATE TABLE embeddings (key text, embedding array)')

In [21]:
%%time
for key, emb in embeddings():
    cursor.execute('INSERT INTO embeddings (key, embedding) VALUES (?, ?)', [key, emb])

CPU times: user 154 ms, sys: 119 µs, total: 154 ms
Wall time: 151 ms


In [22]:
%%time
for key, emb in embeddings():
    cursor.execute('SELECT * FROM embeddings WHERE key=?', (key,))
    data = cursor.fetchone()

CPU times: user 259 ms, sys: 119 µs, total: 259 ms
Wall time: 258 ms


# Sample some data

In [32]:
write_times = []
read_times = []
# counts = [500, 1000, 5000, 10000, 50000, 100000]
counts = numpy.logspace(2,5,dtype='int')

for c in counts:
    cursor.execute('DROP TABLE embeddings')
    cursor.execute('CREATE TABLE embeddings (key text, embedding array)')

    start_time_write = time.time()
    for key, emb in embeddings(c):
        cursor.execute('INSERT INTO embeddings (key, embedding) VALUES (?, ?)', [key, emb])
    write_times.append(time.time() - start_time_write)
    
    start_time_read = time.time()
    for key, emb in embeddings():
        cursor.execute('SELECT * FROM embeddings WHERE key=?', (key,))
        data = cursor.fetchone()
    read_times.append(time.time() - start_time_read)
    
print('DONE')

DONE


# Results

In [34]:
plotly.offline.init_notebook_mode(connected=True)
trace = plotly.graph_objs.Scatter(
    x = write_times,
    y = count,
    mode = 'markers'
)
layout = plotly.graph_objs.Layout(title="SQLite3 Write Times",
                xaxis=dict(title='Times'),
                yaxis=dict(title='Embedding Count'))
data = [trace]
fig = plotly.graph_objs.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='jupyter-basic-scatter')

In [35]:
plotly.offline.init_notebook_mode(connected=True)
trace = plotly.graph_objs.Scatter(
    x = read_times,
    y = count,
    mode = 'markers'
)
layout = plotly.graph_objs.Layout(title="SQLite3 Read Times",
                xaxis=dict(title='Times'),
                yaxis=dict(title='Embedding Count'))
data = [trace]
fig = plotly.graph_objs.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='jupyter-basic-scatter')